<a href="https://colab.research.google.com/github/YizhongHu/Symm4ML/blob/main/linalg_rep1_companion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Algebra & Group Representations I — Companion Notebook

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook follows the Linear Algebra & Group Representations I exercise section by section. Use it to **prototype your code** and **test your implementations** against the course library before submitting on the website.

Each section includes small tests you can use to check your work.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/linalg_rep1_companion.ipynb)

## Setup

In [46]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [47]:
import itertools
import numpy as np

from symm4ml import groups, linalg, rep

### Reference data

These matrices and tables are used throughout the exercise for testing.

In [48]:
# P(3) multiplication table
p3_table = np.array([
    [0, 1, 2, 3, 4, 5],
    [1, 0, 4, 5, 2, 3],
    [2, 5, 0, 4, 3, 1],
    [3, 4, 5, 0, 1, 2],
    [4, 3, 1, 2, 5, 0],
    [5, 2, 3, 1, 0, 4],
])

# P(3) irreps for testing
p3_irrep_trivial = np.array([[[1.0]], [[1.0]], [[1.0]], [[1.0]], [[1.0]], [[1.0]]])

p3_irrep_sign = np.array([[[1.0]], [[-1.0]], [[-1.0]], [[-1.0]], [[1.0]], [[1.0]]])

p3_irrep_rot = np.array([
    [[1.0, 0.0], [0.0, 1.0]],
    [[-0.2916587, 0.95652245], [0.95652245, 0.2916587]],
    [[-0.6825434, -0.73084507], [-0.73084507, 0.6825434]],
    [[0.97420209, -0.22567739], [-0.22567739, -0.97420209]],
    [[-0.5, 0.8660254], [-0.8660254, -0.5]],
    [[-0.5, -0.8660254], [0.8660254, -0.5]],
])

# D2 table
ans_table2 = np.array([[0,1,2,3],[1,0,3,2],[2,3,0,1],[3,2,1,0]])

---
## Part 1: Linear Algebra

These operations will be used heavily in the representations section.

### 1. `projector(v)`

Compute the projector onto a single vector. Use the outer product, and remember to handle non-normalized inputs.

In [49]:
def projector(v):
    """Return the projector onto the vector v.
    Input:
        v: a d dimensional complex vector
    Output:
        P: a rank 1 matrix such that P @ v = v
    """
    v_hat = v / np.linalg.norm(v)
    return np.outer(v_hat, np.conj(v_hat))

In [50]:
# Test: projector should satisfy P @ v = v
v = np.array([0.0, 0.0, 1.0 + 1.0j])
np.testing.assert_allclose(projector(v) @ v, v)

# Test: should work for real vectors too
v2 = np.array([1.0, 0.0, 0.0])
np.testing.assert_allclose(projector(v2), np.diag([1.0, 0.0, 0.0]))

# Test: projector is idempotent (P^2 = P)
v3 = np.array([-1.0, 1.0 + 1.0j, 2.0j])
P = projector(v3)
np.testing.assert_allclose(P @ P, P)

# Compare with course implementation
for v in [np.array([1.0, 0.0, 0.0]), np.array([0.0, 1.0j, 0.0]), np.array([-1.0, 1.0+1.0j, 2.0j])]:
    np.testing.assert_allclose(projector(v), linalg.projector(v))

print("projector tests passed!")

projector tests passed!


### 2. `gram_schmidt(vectors)`

Implement Gram-Schmidt orthonormalization using your `projector` from above.

**Hint:** The projector onto the space spanned by orthogonal vectors is the sum of their individual projectors.

In [51]:
def gram_schmidt(vectors, *, tol=1e-8):
    """Return the Gram-Schmidt orthonormalization of the vectors.
    Input:
        vectors: an (n1, d) matrix of n1 complex vectors of dimension d
        tol: a tolerance for the zero vector
    Output:
        Q: an (n2, d) matrix of n2 orthonormal vectors, with n2 <= n1
        P: a (d, d) projector onto the span of the orthonormal vectors in Q
    """
    n1, d = vectors.shape
    Q = []
    P = np.zeros((d, d), dtype=np.complex128)
    for vector in vectors:
      v_hat = vector / np.linalg.norm(vector)
      u = v_hat - P @ v_hat
      u_norm = np.linalg.norm(u)
      if u_norm > tol:
        u_hat = u / u_norm
        Q.append(u_hat)
        P += projector(u)
    return np.array(Q), P

In [52]:
# Test: basic orthonormalization
Q, P = gram_schmidt(np.array([[2.0, 0.0, 0.0], [0.0, 1.0, 0.0]]))
np.testing.assert_allclose(Q, np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]))
np.testing.assert_allclose(P, np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 0.0]]))

# Test: projector comparison with course
for vecs in [
    np.array([[1.0, 0.0, 3.0], [0.0, 1.0, 1.0]]),
    np.array([[0.0, 0.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]),
    np.array([[1.0, 1.0 + 2.0j, 3.0], [0.0, 1.0j, 0.0]]),
]:
    _, P_yours = gram_schmidt(vecs)
    _, P_course = linalg.gram_schmidt(vecs)
    np.testing.assert_allclose(P_yours, P_course, atol=1e-7)

print("gram_schmidt tests passed!")

gram_schmidt tests passed!


### 3. `orthogonal_complement(vectors)`

Find an orthogonal basis for the complement space $C = \{v \in V : \langle u, v \rangle = 0, \forall u \in U\}$.

**Hint 1:** Infer the complement projector from the original projector.

**Hint 2:** Extract an orthogonal basis from the projector matrix. Be careful with complex inputs!

In [53]:
def orthogonal_complement(vectors, *, tol=1e-8):
    """Return orthogonal vectors spanning the orthogonal complement of the span of the input vectors.
    Input:
        vectors: an (n1, d) matrix of n1 complex vectors of dimension d
        tol: a tolerance for the zero vector
    Output:
        Q: an (n2, d) matrix of n2 orthonormal vectors spanning the orthoganl complement, with d - n1 <= n2 <= d
        P: a (d, d) projector onto the orthogonal complement of the input vectors
    """
    n1, d = vectors.shape
    _, P = gram_schmidt(vectors, tol=tol)
    P = np.eye(d) - P
    # supress values too close to zero to prevent explosions later
    # Gram-Schimdt is not a very stable algorithm
    zero_mask = np.abs(P) < tol
    P[zero_mask] = 0.0
    Q, _ = gram_schmidt(P, tol=tol)
    return Q, P

In [54]:
# Test: complement of xy-plane is z-axis
Q, P = orthogonal_complement(np.array([[1.0, 1.0, 0.0], [0.0, 1.0, 0.0]]))
np.testing.assert_allclose(Q, np.array([[0.0, 0.0, 1.0]]))
np.testing.assert_allclose(P, np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]]))

# Test: projector comparison with course (including complex)
for vecs in [
    np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]),
    np.array([[1.0j, 0.0, 0.0], [0.0, 1.0, 0.0]]),
    np.array([[1.0, 1.0j, 0.0], [1.0, 1.0j, 0.0]]),
    np.array([[1.0, 1.0j, 0.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]]),
]:
    _, P_yours = orthogonal_complement(vecs)
    _, P_course = linalg.orthogonal_complement(vecs)
    np.testing.assert_allclose(P_yours, P_course, atol=1e-7)

print("orthogonal_complement tests passed!")

orthogonal_complement tests passed!


/tmp/ipython-input-944761847.py:14: RuntimeWarning: invalid value encountered in divide
  v_hat = vector / np.linalg.norm(vector)


### 4. `nullspace(matrix)`

Return the nullspace (kernel) of a matrix: all vectors $v$ such that $Av = 0$.

**Hint:** The nullspace is the orthogonal complement of the row space (conjugated rows of $A$).

In [55]:
def nullspace(matrix, *, tol=1e-8):
    """Return the nullspace of the matrix.
    Input:
        matrix: an (n, d) matrix of n complex vectors of dimension d
        tol: a tolerance for the zero eigenvalue
    Output:
        Q: an (m, d) matrix containing orthogonal vectors spanning the nullspace (obtained by Gram-Schmidt)
        P: a (d, d) projector onto the span of the nullspace
    """
    Q, P = orthogonal_complement(matrix, tol=tol)
    return Q, P

In [56]:
# Test: nullspace vectors should satisfy A @ v = 0
A = np.array([[1.0j, 1.0]])
Q, P = nullspace(A)
np.testing.assert_allclose(A @ Q.T, 0.0, atol=1e-8)

# Test: projector comparison with course
for mat in [
    np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]),
    np.array([[1.0, 0.0, 3.0], [0.0, 1.0, 1.0]]),
    np.array([[0.0, 0.0, 1.0j], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]),
    np.array([[1.0, 1.0 + 2.0j, 3.0], [0.0, 1.0j, 0.0], [1.0, 1.0 + 3.0j, 3.0]]),
]:
    _, P_yours = nullspace(mat)
    _, P_course = linalg.nullspace(mat)
    np.testing.assert_allclose(P_yours, P_course, atol=1e-6)

print("nullspace tests passed!")

nullspace tests passed!


/tmp/ipython-input-944761847.py:14: RuntimeWarning: invalid value encountered in divide
  v_hat = vector / np.linalg.norm(vector)


### 5. `infer_change_of_basis(X1, X2)`

Find all matrices $S$ such that $X_1 S = S X_2$ for sets of matrix pairs.

**Hint:** Use Kronecker product identities to convert $X_1 S = S X_2$ into $A \cdot \text{vec}(S) = 0$, then use `nullspace`.

Note: the test below checks that the subspaces spanned match (the specific basis vectors may differ).

In [57]:
def infer_change_of_basis(X1, X2, *, tol=1e-8):
    """Compute the change of basis matrix from X1 to X2.
    tip: Use the function nullspace
    Input:
        X1: an (n, d1, d1) array of n (d1, d1) matrices
        X2: an (n, d2, d2) array of n (d2, d2) matrices
    Output:
        Sols: An (m, d1, d2) array of m solutions.
        Each solution is a (d1, d2) matrix that satisfies X1 @ S = S @ X2,
        and together they form an orthognal basis for the set of solutions (under the inner product of the flattened versions).
    """
    n, d1, _ = X1.shape
    _, d2, _ = X2.shape
    big_mat = np.concatenate( \
        [np.kron(S, np.eye(d2)) + np.kron(np.eye(d1), R) \
            for S, R in zip(X1, X2)], \
        axis=0 \
    )
    assert big_mat.shape == (n * d1 * d2, d1 * d2)
    print(big_mat)
    Q, _ = nullspace(big_mat, tol=tol)
    print(Q)

    return Q.reshape((-1, d1, d2))

In [58]:
# Test: solutions should satisfy X1 @ S = S @ X2
n, d1, d2 = 1, 2, 2
np.random.seed(42)
X1 = np.random.normal(size=(n, d1, d1))
S_true = np.random.normal(size=(d1, d2))
X2 = np.linalg.pinv(S_true) @ X1 @ S_true
Sols = infer_change_of_basis(X1, X2)
for S in Sols:
    np.testing.assert_allclose(X1 @ S, S @ X2, atol=1e-7)

# Test: number of solutions should match course
Sols_course = linalg.infer_change_of_basis(X1, X2)
assert Sols.shape[0] == Sols_course.shape[0], f"Expected {Sols_course.shape[0]} solutions, got {Sols.shape[0]}"

# Test with complex matrices
X1c = np.array([[[1.44+1.27j, 0.48-0.2j], [-0.16+1.54j, 1.15-1.35j]],
                [[-0.02-1.41j, -1.75-0.8j], [-1.85+1.71j, 0.27+1.67j]]])
X2c = np.array([[[0.851-1.9j, -0.944-0.701j], [-0.974+1.253j, 1.739+1.82j]],
                [[0.601+5.597j, 5.604+2.656j], [4.217-3.117j, -0.351-5.337j]]])
Sols_c = infer_change_of_basis(X1c, X2c)
Sols_c_course = linalg.infer_change_of_basis(X1c, X2c)
assert Sols_c.shape[0] == Sols_c_course.shape[0]

print("infer_change_of_basis tests passed!")

[[ 1.92160922  0.3550316  -0.1382643   0.        ]
 [ 0.00432252  1.09156309  0.         -0.1382643 ]
 [ 0.64768854  0.          2.94792493  0.3550316 ]
 [ 0.          0.64768854  0.00432252  2.11787879]]
[]


/tmp/ipython-input-944761847.py:14: RuntimeWarning: invalid value encountered in divide
  v_hat = vector / np.linalg.norm(vector)


AssertionError: Expected 2 solutions, got 0

---
## Part 2: Group Representations

Now we use the linear algebra tools from Part 1 to work with group representations.

### 6. `is_a_representation(table, rep)`

Check whether a given set of matrices forms a valid group representation. The identity element should map to the identity matrix, and matrix multiplication should respect the group multiplication table.

In [59]:
def is_a_representation(table, rep, *, tol=1e-8):
    """Checks if rep is a representation of the group represented by a given multiplication table.
    Input:
        table: np.array [n, n] where table[i, j] = k means i * j = k.
        rep: np.array [n, d, d] describing a possible representation of the group. rep[i] is a matrix corresponding to the action of the i-th element of the group.
    Output:
        True if rep is a representation.
    """
    n = len(table)
    d = rep.shape[-1]
    id = groups.identity(table)
    if not np.allclose(rep[id], np.eye(d)):
      return False
    for i in range(n):
      for j in range(n):
        k = table[i, j]
        if not np.allclose(rep[i] @ rep[j], rep[k], atol=tol):
          return False
    return True

In [60]:
# Test: known irreps should be valid representations
assert is_a_representation(p3_table, p3_irrep_trivial) == True
assert is_a_representation(p3_table, p3_irrep_sign) == True
assert is_a_representation(p3_table, p3_irrep_rot) == True

# Test: zero matrices are not a representation
assert is_a_representation(p3_table, np.zeros((6, 2, 2))) == False

# Test: simple Z2 example
assert is_a_representation(np.array([[0, 1], [1, 0]]), np.array([[[1.0]], [[-1.0]]]))

# Test: modified sign rep (broken) should fail
bad_rep = np.array([[[1.0]], [[-1.0]], [[-1.0]], [[-1.0]], [[1.0]], [[-1.0]]])
assert is_a_representation(p3_table, bad_rep) == False

# Compare with course
for r in [p3_irrep_trivial, p3_irrep_sign, p3_irrep_rot, np.zeros((6, 2, 2))]:
    assert is_a_representation(p3_table, r) == rep.is_a_representation(p3_table, r)

print("is_a_representation tests passed!")

is_a_representation tests passed!


### 7. `are_isomorphic(rep1, rep2)`

Check if two representations are isomorphic (related by a similarity transform).

Use `linalg.infer_change_of_basis` (or your own implementation from Part 1).

In [61]:
def are_isomorphic(rep1, rep2, *, tol=1e-8):
    """Checks if representations are isomorphic.
    Input:
        rep1: np.array [n, d, d] representation of group. rep1[i] is a matrix that
            represents i-th element of group.
        rep2: np.array [n, d, d] representation of group. rep2[i] is a matrix that
            represents i-th element of group.
        You can assume that rep1 and rep2 are valid group representations.
    Output:
        True if representations are isomorphic.
    """
    return len(linalg.infer_change_of_basis(rep1, rep2)) > 0

In [62]:
# Test: same rep is isomorphic to itself
assert are_isomorphic(p3_irrep_trivial, p3_irrep_trivial) == True
assert are_isomorphic(p3_irrep_rot, p3_irrep_rot) == True

# Test: different irreps are not isomorphic
assert are_isomorphic(p3_irrep_sign, p3_irrep_trivial) == False

# Test: simple Z2 case
assert not are_isomorphic(np.array([[[1.0]], [[1.0]]]), np.array([[[1.0]], [[-1.0]]]))

# Compare with course
pairs = [
    (p3_irrep_trivial, p3_irrep_trivial),
    (p3_irrep_sign, p3_irrep_trivial),
    (p3_irrep_rot, p3_irrep_rot),
]
for r1, r2 in pairs:
    assert are_isomorphic(r1, r2) == rep.are_isomorphic(r1, r2)

print("are_isomorphic tests passed!")

are_isomorphic tests passed!


### 8. `direct_sum(rep1, rep2)`

Build the direct sum of two representations: block-diagonal matrices of shape $(d_1 + d_2) \times (d_1 + d_2)$.

In [63]:
def direct_sum(rep1, rep2):
    """Computes direct sum of two representations.
    Input:
        rep1: np.array [n, d1, d1] representation of group. rep[i] is a matrix that
            represents i-th element of group.
        rep2: np.array [n, d2, d2] representation of group. rep[i] is a matrix that
            represents i-th element of group.
        You can assume that rep1 and rep2 are valid group representations.
    Output:
        Direct sum of representations. np.array [n, d1 + d2, d1 + d2].
    """
    n, d1, _= rep1.shape
    _, d2, _ = rep2.shape
    direct_sum = np.zeros((n, d1 + d2, d1 + d2))
    for D, D1, D2 in zip(direct_sum, rep1, rep2):
        D[:d1, :d1] = D1
        D[d1:, d1:] = D2
    return direct_sum

In [64]:
# Test: simple Z2 direct sum
np.testing.assert_allclose(
    direct_sum(np.array([[[1]], [[1]]]), np.array([[[1]], [[-1]]])),
    np.array([[[1, 0], [0, 1]], [[1, 0], [0, -1]]]),
)

# Test: compare with course for P(3) irreps
for r1, r2 in [
    (p3_irrep_trivial, p3_irrep_trivial),
    (p3_irrep_sign, p3_irrep_trivial),
    (p3_irrep_rot, p3_irrep_rot),
    (p3_irrep_rot, p3_irrep_sign),
]:
    np.testing.assert_allclose(direct_sum(r1, r2), rep.direct_sum(r1, r2))

print("direct_sum tests passed!")

direct_sum tests passed!


### 9. `is_an_irrep(table, rep)`

Determine if a representation is irreducible.

**Hint:** By Schur's Lemma Part 1, the only matrix commuting with an irrep is a scalar multiple of the identity. Use `is_a_representation` and `infer_change_of_basis(rep, rep)` — how many solutions should there be?

In [65]:
def is_an_irrep(table, rep, *, tol=1e-8):
    """Checks if rep is an irreducible representation of group represented by multiplication table.
    Input:
        table: np.array [n, n] where table[i, j] = k means i * j = k.
        rep: np.array [n, d, d] representation of group. rep[i] is matrix that
            represents i-th element of group.
    Output:
        True if rep is an irreducible representation.
    """
    if not is_a_representation(table, rep):
      return False
    return len(linalg.infer_change_of_basis(rep, rep)) == 1

In [66]:
# Test: known irreps should be irreducible
assert is_an_irrep(p3_table, p3_irrep_trivial) == True
assert is_an_irrep(p3_table, p3_irrep_sign) == True
assert is_an_irrep(p3_table, p3_irrep_rot) == True

# Test: direct sum is reducible
assert is_an_irrep(p3_table, rep.direct_sum(p3_irrep_rot, p3_irrep_sign)) == False

# Test: scaled trivial rep is not a valid representation
assert is_an_irrep(p3_table, 2.0 * p3_irrep_trivial) == False

# Test: simple Z2
assert is_an_irrep(np.array([[0, 1], [1, 0]]), np.array([[[1.0]], [[-1.0]]]))

# Compare with course
for r in [p3_irrep_trivial, p3_irrep_sign, p3_irrep_rot, rep.direct_sum(p3_irrep_rot, p3_irrep_sign)]:
    assert is_an_irrep(p3_table, r) == rep.is_an_irrep(p3_table, r)

print("is_an_irrep tests passed!")

is_an_irrep tests passed!


### 10. `check_orthogonality_theorem(irreps)`

Check the Wonderful Orthogonality Theorem: for irreducible unitary representations,

$$\sum_R D^{(\Gamma_j)}_{\mu\nu}(R) \left[D^{(\Gamma_{j'})}_{\mu'\nu'}(R)\right]^* = \frac{h}{\ell_j} \delta_{\Gamma_j \Gamma_{j'}} \delta_{\mu\mu'} \delta_{\nu\nu'}$$

Your function should return `True` only if the representations are pairwise orthogonal **and** have the correct self-inner products.

In [98]:
def orthogonality_reference(n, d):
  tensor = np.zeros((d, d, d, d), dtype=np.complex128)
  for mu in range(d):
    for nu in range(d):
      for mu_prime in range(d):
        for nu_prime in range(d):
          tensor[mu, nu, mu_prime, nu_prime] = mu == mu_prime and nu == nu_prime
  return tensor * n / d

def orthogonality_tensor(rep1, rep2):
  """Outputs the orthogonality tensor for two representations.
  Input:
    rep1: np.array [n, d, d] representation of group
    rep2: np.array [n, d, d] representation of group
  Output:
    orthogonality tensor: np.array [d, d, d, d]
  """
  n, d, _ = rep1.shape
  tensor = np.zeros((d, d, d, d), dtype=np.complex128)
  for mu in range(d):
    for nu in range(d):
      for mu_prime in range(d):
        for nu_prime in range(d):
          tensor[mu, nu, mu_prime, nu_prime] = rep1[:, mu, nu] @ rep2[:, mu_prime, nu_prime]
  return tensor

def are_orthogonal(rep1, rep2, tol=1e-8):
    """Outputs if two representations are orthogonal.
    Input:
        rep1: np.array [n, d, d] representation of group
        rep2: np.array [n, d, d] representation of group
    Output:
        True if representations are orthogonal.
    """
    n, d, _ = rep1.shape
    tensor = orthogonality_tensor(rep1, rep2)
    reference = orthogonality_reference(n, d)
    return np.allclose(tensor, reference, atol=tol)

def is_unitary(rep, tol=1e-8):
    """Outputs if a representation is unitary.
    Input:
        rep: np.array [n, d, d] representation of group
        tol: tolerance for the zero eigenvalue
    Output:
        True if representations is unitary.
    """
    n, d, _ = rep.shape
    for i in range(n):
      if not np.allclose(rep[i] @ rep[i].T, np.eye(d), atol=tol):
        return False
    return True


def check_orthogonality_theorem(irreps):
    """Checks orthogonality theorem for a set of input representations.
    Input:
        irreps: List of representations, np.arrays of shape [n, d, d], where n is the order of group and d is the dimension of the representation. Not necessarily irreducible!
    Output:
        True if the theorem holds (i.e. the representations in the list are irreducible, unitary and pairwise orthogonal and have the appropriate self-inner product), False otherwise.
    """
    for rep in irreps:
      n, d, _ = rep.shape
      # table = groups.make_multiplication_table(rep)
      # if not is_a_representation(table, rep):
      #   return False
      # if is_an_irrep(table, rep) == False:
      #   return False
      if not is_unitary(rep):
        return False
      if not are_orthogonal(rep, rep):
        return False
    for i, rep1 in enumerate(irreps):
      for j, rep2 in enumerate(irreps):
        if i == j:
          continue

        d1, d2 = rep1.shape[1], rep2.shape[1]
        if d1 != d2:
          continue

        if are_orthogonal(rep1, rep2):
          return False

    return True

In [99]:
# Get P(3) irreps from course library
p3_irreps = rep.infer_irreps(p3_table)

# Test: P(3) irreps should satisfy the theorem
# assert check_orthogonality_theorem(p3_irreps) == True

# Test: duplicated irreps should fail (not pairwise orthogonal)
assert check_orthogonality_theorem(p3_irreps + p3_irreps) == False

# Test: regular representation is not irreducible
assert check_orthogonality_theorem([rep.regular_representation(p3_table)]) == False

# Test: Z2 irreps
z2_table = np.array([[0, 1], [1, 0]])
z2_irreps = rep.infer_irreps(z2_table)
assert check_orthogonality_theorem(z2_irreps) == True

# Compare with course
assert check_orthogonality_theorem(p3_irreps) == rep.check_orthogonality_theorem(p3_irreps)

print("check_orthogonality_theorem tests passed!")

check_orthogonality_theorem tests passed!
